In [15]:
import cv2
import numpy as np
from collections import deque

In [23]:
camera = cv2.VideoCapture(0)
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])
print('blueUpper:',blueUpper)
kernel = np.ones((5, 5), np.uint8)
pts = deque(maxlen=512)
# cv2.imshow('video',img)
while True:
    check,frame = camera.read()
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)
    
#     image,cnts,hierarchy = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts, heir = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    if len(cnts)>0:
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        if cv2.contourArea(cnt) > 200:
            ((x, y), radius) = cv2.minEnclosingCircle(cnt)
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            M = cv2.moments(cnt)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            pts.appendleft(center)
            for i in range(1, len(pts)):
                if pts[i - 1] is None or pts[i] is None:
                    continue
                cv2.line(frame, pts[i - 1], pts[i], (0, 255,255), 7)          
    else:
        pts = deque(maxlen=512)
    cv2.imshow("video",frame)
    k = cv2.waitKey(10)
    if k == 27 :
        break
camera.release()
cv2.destroyAllWindows()
    

blueUpper: [140 255 255]
